**Evaluation Metrics**

In [ ]:
# Import necessary libraries
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import glob
import os
from google.colab.patches import cv2_imshow
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

In [ ]:
# Specify the path to the Excel file in Google Drive

file_path = '/content/drive/MyDrive/Results.xlsx'

# Load data from the Excel sheet. This data consists of the frame scores of test data which has both predicted values and actual values

df = pd.read_excel(file_path, sheet_name='Dataset')
Actual = df['Actual'].to_numpy()
Predicted = df['Predicted'].to_numpy()

In [ ]:
# Confusion Matrix
threshold = 0.6
y_pred = (Predicted > threshold).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(Actual, y_pred)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = cm.ravel()

# Print TP, TN, FP, FN
print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")

# Calculate precision, recall, specificity and F1 score
precision = precision_score(Actual, y_pred)
recall = recall_score(Actual, y_pred)
f1 = f1_score(Actual, y_pred)
specificity = TN / (TN + FP)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")

# Update font properties with an alternative serif font
plt.rcParams.update({'font.size': 16, 'font.family': 'serif', 'font.serif': 'DejaVu Serif'})

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
ax = sns.heatmap(cm, annot=True, fmt='d', cmap="Purples", cbar=True,
                 xticklabels=['Abnormal (0)', 'Normal (1)'],
                 yticklabels=['Abnormal (0)', 'Normal (1)'])

# Set axis labels
plt.xlabel('Predicted Event', color='black')
plt.ylabel('Actual Event', color='black')

# Set tick labels color to black
ax.tick_params(axis='both', colors='black')

plt.title('Dataset', fontsize=16)
plt.show()


In [ ]:
#ROC

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (16,9)
%matplotlib inline

P = sum(Actual)
N = len(Actual) - P
FPR = []
TPR = []
# Iterate thresholds from 0.0 to 1.0
thresholds = np.arange(0.0, 1.01, 0.1)
# array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9. 1.0 ])

# iterate through all thresholds and determine fraction of true positives
# and false positives found at this threshold
for thresh in thresholds:
    FP=0
    TP=0
    thresh = round(thresh,2)
    for i in range(len(Predicted)):
        if (Predicted[i] >= thresh):
            if Actual[i] == 1:
                TP = TP + 1
            if Actual[i] == 0:
                FP = FP + 1
    FPR.append(FP/N)
    TPR.append(TP/P)
print('TPR:', TPR)
print('FPR:', FPR)

#AUC
auc = -1 * np.trapz(TPR, FPR)

plt.plot(FPR, TPR, linestyle='--', marker='o', color='darkorange', lw = 2, label='Test_Dataset', clip_on=False)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve; AUC = %.4f'%auc)
plt.legend(loc="lower right")
plt.show()